In [1]:
import sys
sys.path.append("../")
from plotting import CandlePlot
import pandas as pd

In [2]:
from technicals.patterns import apply_patterns

In [3]:
df = pd.read_pickle("../data/GBP_JPY_H1.pkl")

In [4]:
df_an = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [5]:
df_an.tail()

,time,mid_o,mid_h,mid_l,mid_c
37219,2021-12-30 19:00:00+00:00,155.312,155.444,155.274,155.444
37220,2021-12-30 20:00:00+00:00,155.442,155.470,155.289,155.312
37221,2021-12-30 21:00:00+00:00,155.317,155.382,155.286,155.382
37222,2021-12-30 22:00:00+00:00,155.353,155.381,155.282,155.329
37223,2021-12-30 23:00:00+00:00,155.336,155.360,155.284,155.310


In [6]:
df_an = apply_patterns(df_an)

In [12]:
HANGING_MAN_BODY = 15.0
HANGING_MAN_HEIGHT = 75.0
SHOOTING_STAR_HEIGHT = 25.0
SPINNING_TOP_MIN = 40.0
SPINNING_TOP_MAX = 60.0
MARUBOZU = 98.0
ENGULFING_FACTOR = 1.1

apply_marubozu = lambda x: x.body_perc > MARUBOZU
def apply_hanging_man(row):
    if row.body_bottom_perc > HANGING_MAN_HEIGHT:
        if row.body_perc < HANGING_MAN_BODY:
            return True
    return False

def apply_shooting_star(row):
    if row.body_top_perc < SHOOTING_STAR_HEIGHT:
        if row.body_perc < HANGING_MAN_BODY:
            return True
    return False

def apply_spinning_top(row):
    if row.body_top_perc < SPINNING_TOP_MAX and row.body_bottom_perc > SPINNING_TOP_MIN:
        if row.body_perc < HANGING_MAN_BODY:
            return True
    return False

def apply_engulfing(row):
    if row.direction != row.direction_prev:
        if row.body_size > row.body_size_prev * ENGULFING_FACTOR:
            return True
    return False

TWEEZER_BODY = 15.0
TWEEZER_HL = 0.01
TWEEZER_TOP_BODY = 40.0
TWEEZER_BOTTOM_BODY = 60.0

def apply_tweezer_top(row):
    if abs(row.body_size_change) < TWEEZER_BODY:
        if row.direction == -1 and row.direction != row.direction_prev:
            if abs(row.low_change) < TWEEZER_HL and abs(row.high_change) < TWEEZER_HL:
                if row.body_top_perc < TWEEZER_TOP_BODY:
                    return True
    return False

def apply_tweezer_bottom(row):
    if abs(row.body_size_change) < TWEEZER_BODY:
        if row.direction == 1 and row.direction != row.direction_prev:
            if abs(row.low_change) < TWEEZER_HL and abs(row.high_change) < TWEEZER_HL:
                if row.body_bottom_perc > TWEEZER_BOTTOM_BODY:
                    return True
    return False

MORNING_STAR_PREV2_BODY = 90.0
MORNING_STAR_PREV_BODY = 10.0

def apply_morning_star(row, direction=1):
    if row.body_perc_prev_2 > MORNING_STAR_PREV2_BODY:
        if row.body_perc_prev < MORNING_STAR_PREV_BODY:
            if row.direction == direction and row.direction_prev_2 != direction:
                if direction == 1:
                    if row.mid_c > row.midpoint_prev_2:
                        return True
                else:
                    if row.mid_c < row.midpoint_prev_2:
                        return True
    return False

In [13]:
df_an['body_size_prev'] = df_an.body_size.shift(1)
df_an['direction_prev'] = df_an.direction.shift(1)
df_an['direction_prev_2'] = df_an.direction.shift(2)
df_an['body_perc_prev'] = df_an.body_perc.shift(1)
df_an['body_perc_prev_2'] = df_an.body_perc.shift(2)
df_an['HANGING_MAN'] = df_an.apply(apply_hanging_man, axis=1)
df_an['SHOOTING_STAR'] = df_an.apply(apply_shooting_star, axis=1)
df_an['SPINNING_TOP'] = df_an.apply(apply_spinning_top, axis=1)
df_an['MARUBOZU'] = df_an.apply(apply_marubozu, axis=1)
df_an['ENGULFING'] = df_an.apply(apply_engulfing, axis=1)
df_an['TWEEZER_TOP'] = df_an.apply(apply_tweezer_top, axis=1)
df_an['TWEEZER_BOTTOM'] = df_an.apply(apply_tweezer_bottom, axis=1)
df_an['MORNING_STAR'] = df_an.apply(apply_morning_star, axis=1)
df_an['EVENING_STAR'] = df_an.apply(apply_morning_star, axis=1, direction = -1)

In [14]:
import plotly.graph_objects as go

In [18]:
MARKER = '#0066FF'
dfp = df_an.iloc[26200:26300]
cp = CandlePlot(dfp, candles=True)
df_temp = cp.df_plot[cp.df_plot.EVENING_STAR == True]
cp.fig.add_trace(go.Candlestick(
                x = df_temp.sTime,
                open = df_temp.mid_o,
                high = df_temp.mid_h,
                low = df_temp.mid_l,
                close = df_temp.mid_c,
                line = dict(width=1), opacity=1,
                increasing_fillcolor = MARKER,
                decreasing_fillcolor = MARKER,
                increasing_line_color = MARKER,
                decreasing_line_color = MARKER
            ))
cp.fig.update_yaxes(
            gridcolor = "#1f292f"
)
cp.fig.update_xaxes(
    gridcolor = "#1f292f",
    rangeslider = dict(visible = False),
    nticks = 5
)
cp.fig.update_layout(
    width = 825,
    height = 400,
    margin = dict(l=10, r=10, b=10, t=10),
    paper_bgcolor="#2c303c",
    plot_bgcolor="#2c303c",
    font = dict(size=8, color = "#e1e1e1")
)